In [ ]:
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql import functions as F, types as T
from pyspark.rdd import RDD

import os

home_dir = os.path.expanduser("~")
print(home_dir)

In [ ]:
import pyspark
print(pyspark.__version__)

In [ ]:
spark: SparkSession = SparkSession.builder \
    .appName("GPU RAPIDS SQL Example") \
    .master("local[*]") \
    .config("spark.plugins", "com.nvidia.spark.SQLPlugin") \
    .config("spark.rapids.sql.enabled", "true") \
    .config("spark.rapids.sql.explain", "ALL") \
    .config("spark.executor.resource.gpu.amount", "1") \
    .config("spark.driver.extraClassPath", f"{home_dir}/rapids_jars/rapids-4-spark_2.12-25.04.0.jar:{home_dir}/rapids_jars/cudf-25.04.0-cuda12.jar") \
    .config("spark.executor.extraClassPath", f"{home_dir}/rapids_jars/rapids-4-spark_2.12-25.04.0.jar:{home_dir}/rapids_jars/cudf-25.04.0-cuda12.jar") \
    .getOrCreate()

sc = spark.sparkContext
print(f"Monitor cluster at: {sc.uiWebUrl}")

### Generate and process resilient distributed dataset (RDD) from an array on the driver

In [ ]:
data = [1, 2, 3, 4, 5]
rdd: RDD[int] = sc.parallelize(data)
result = rdd.map(lambda x: x * 2).collect()
print(result)  # Output: [2, 4, 6, 8, 10]

### Generate and process a DataFrame from a .csv file on the driver 

In [ ]:
higgs_data: DataFrame = spark.read.csv(
    "../data/HIGGS.csv",
    header=False,
    inferSchema=True
)
higgs_data.printSchema()

In [ ]:
(
    higgs_data
    .agg(
        F.mean(F.col("_c0")).alias("mean_c0"),
        F.stddev(F.col("_c0")).alias("stddev_c0"),
    )
    .toPandas()
)

### Free cluster resources

In [ ]:
spark.stop()